# Pre-Processing of Bank Dataset

In [1]:
# Environment setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

In [8]:
# custom library
!pip install --force-reinstall --no-cache git+https://github.com/junclemente/jcds.git@develop

from jcds import reports as jrep
from jcds import eda as jeda

  Cloning https://github.com/junclemente/jcds.git (to revision develop) to /tmp/pip-req-build-lru1h3hv
  Running command git clone --filter=blob:none --quiet https://github.com/junclemente/jcds.git /tmp/pip-req-build-lru1h3hv
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  branch 'develop' set up to track 'origin/develop'.
  Resolved https://github.com/junclemente/jcds.git to commit 4ed4497ad47fafc4f1f965e8d246a32eaabfbf78
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.9/831.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 32.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.9 M

In [3]:
datasets = Path("../datasets")

df = pd.read_csv(datasets / "bank-additional-full.csv", sep=";")
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
jrep.data_cardinality(df, show_columns=True)

CARDINALITY REPORT

Total columns analyzed: 21

[BINARY COLUMNS]
There are 2 binary columns.
 * Columns: ['contact', 'y']
There are 0 binary with nan.

[CONSTANT/NEAR CONSTANT COLUMNS]
There are 0 constant columns.
There are 1 near-constant columns with >= 95% of values being the same.
 * Columns: ['pdays']

[LOW CARDINALITY CATEGORICAL COLUMNS]
 * There are 10 low cardinality columns with <= 10 unique values.
Columns:
 * marital: 4 unique values
 * education: 8 unique values
 * default: 3 unique values
 * housing: 3 unique values
 * loan: 3 unique values
 * contact: 2 unique values
 * month: 10 unique values
 * day_of_week: 5 unique values
 * poutcome: 3 unique values
 * y: 2 unique values

[HIGH CARDINALITY CATEGORICAL COLUMNS]
 * There are 0 high cardinality variables with >=90% unique values.


In [5]:
jeda.detect_outliers_iqr(df)

{'age': 469,
 'duration': 2963,
 'campaign': 2406,
 'pdays': 1515,
 'previous': 5625,
 'emp.var.rate': 0,
 'cons.price.idx': 0,
 'cons.conf.idx': 447,
 'euribor3m': 0,
 'nr.employed': 0}

In [7]:
jeda.plot_outlier_boxplots(df)

AttributeError: module 'jcds.eda' has no attribute 'plot_outlier_boxplots'